# INF582 AXA Challenge

## Initialisation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Load main librairies
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import itertools
import random

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import __version__
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import train_test_split

pd.set_option('display.max_columns', 500)

## Load data

In [8]:
# Load data
submission = pd.read_csv("data/submission.txt", sep='\t')
training_data = pd.read_csv("data/train_2011_2012.csv", sep=';', nrows=10000,
                            na_values=['A Définir', 'A DEFINIR', '9999-12-31 00:00:00.000'],
                            #usecols=['DATE','WEEK_END','DAY_WE_DS','TPER_TEAM','ASS_ASSIGNMENT','CSPL_RECEIVED_CALLS']
                            )
training_data.head()

,DATE,DAY_OFF,DAY_DS,WEEK_END,DAY_WE_DS,TPER_TEAM,TPER_HOUR,SPLIT_COD,ACD_COD,ACD_LIB,ASS_SOC_MERE,ASS_DIRECTORSHIP,ASS_ASSIGNMENT,ASS_PARTNER,ASS_POLE,ASS_BEGIN,ASS_END,ASS_COMENT,CSPL_I_STAFFTIME,CSPL_I_AVAILTIME,CSPL_I_ACDTIME,CSPL_I_ACWTIME,CSPL_I_ACWOUTTIME,CSPL_I_ACWINTIME,CSPL_I_AUXOUTTIME,CSPL_I_AUXINTIME,CSPL_I_OTHERTIME,CSPL_ACWINCALLS,CSPL_ACWINTIME,CSPL_AUXINCALLS,CSPL_AUXINTIME,CSPL_ACWOUTCALLS,CSPL_ACWOUTIME,CSPL_ACWOUTOFFCALLS,CSPL_ACWOUTOFFTIME,CSPL_AUXOUTCALLS,CSPL_AUXOUTTIME,CSPL_AUXOUTOFFCALLS,CSPL_AUXOUTOFFTIME,CSPL_INFLOWCALLS,CSPL_ACDCALLS,CSPL_ANSTIME,CSPL_HOLDCALLS,CSPL_HOLDTIME,CSPL_HOLDABNCALLS,CSPL_TRANSFERED,CSPL_CONFERENCE,CSPL_ABNCALLS,CSPL_ABNTIME,CSPL_ABNCALLS1,CSPL_ABNCALLS2,CSPL_ABNCALLS3,CSPL_ABNCALLS4,CSPL_ABNCALLS5,CSPL_ABNCALLS6,CSPL_ABNCALLS7,CSPL_ABNCALLS8,CSPL_ABNCALLS9,CSPL_ABNCALLS10,CSPL_OUTFLOWCALLS,CSPL_OUTFLOWTIME,CSPL_MAXINQUEUE,CSPL_CALLSOFFERED,CSPL_I_RINGTIME,CSPL_RINGTIME,CSPL_RINGCALLS,CSPL_NOANSREDIR,CSPL_MAXSTAFFED,CSPL_ACWOUTADJCALLS,CSPL_AUXOUTADJCALLS,CSPL_DEQUECALLS,CSPL_DEQUETIME,CSPL_DISCCALLS,CSPL_DISCTIME,CSPL_INTRVL,CSPL_INCOMPLETE,CSPL_ACCEPTABLE,CSPL_SERVICELEVEL,CSPL_ACDAUXOUTCALLS,CSPL_SLVLABNS,CSPL_SLVLOUTFLOWS,CSPL_RECEIVED_CALLS,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_ACWTIME,CSPL_ACDTIME
0,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,855,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,5400,4812,0,0,0,0,0,0,543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,30,0,0,30,0,0,0,0,0,0,0,0
1,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1056,1,Entity1 G3RV6,Entity2 Belgique,Administrative et Financière,Finances PCX,NaN,NaN,2010-01-01 00:00:00.000,NaN,NaN,1800,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0
2,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1057,1,Entity1 G3RV6,Entity2 Belgique,Administrative et Financière,Finances PCX,NaN,NaN,2010-01-01 00:00:00.000,NaN,NaN,1800,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0
3,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1587,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,12600,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,30,0,0,15,0,0,0,0,0,0,0,0
4,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1589,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,7200,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0


In [58]:
meteo2012 = pd.read_csv("data/meteo_2012.csv",nrows=10000,
                    header=None,
                    usecols=[0,1,3,6]
                   )

meteo2012.rename(columns={0: 'DATE', 1: 'DEPT',3:'TEMP',6:'RAIN'}, inplace=True)
meteo2012 = meteo2012[meteo2012[['DEPT']].apply(lambda x: x[0].isdigit(), axis=1)]
meteo2012 = meteo2012[~meteo2012.DEPT.isin(['00'])]

meteo2011 = pd.read_csv("data/meteo_2011.csv",nrows=10000,
                    header=None,
                    usecols=[0,1,3,6]
                   )
meteo2011.rename(columns={0: 'DATE', 1: 'DEPT',3:'TEMP',6:'RAIN'}, inplace=True)
meteo2011 = meteo2011[meteo2011[['DEPT']].apply(lambda x: x[0].isdigit(), axis=1)]
meteo2011 = meteo2011[~meteo2011.DEPT.isin(['00'])]

frames=[meteo2012,meteo2011]

meteo=pd.concat(frames)

meteo.head(200)



,DATE,DEPT,TEMP,RAIN
0,2012-01-01 00:00,44,12.9,0.0
1,2012-01-01 00:00,59,12.2,0.0
2,2012-01-01 00:00,59,12.1,0.0
3,2012-01-01 00:00,60,12.3,0.0
4,2012-01-01 00:00,60,12.1,0.0
5,2012-01-01 00:00,61,12.5,0.0
6,2012-01-01 00:00,62,10.8,0.0
7,2012-01-01 00:00,62,11.2,0.0
8,2012-01-01 00:00,62,11.0,0.0
9,2012-01-01 00:00,63,13.5,0.0


## Clean data

In [4]:
# Remove non usefull rows
training_data = training_data[training_data.ASS_ASSIGNMENT.isin(submission.ASS_ASSIGNMENT.unique())]

In [5]:
# Aggregate the calls
training_data = training_data.groupby([col for col in training_data.columns if not col == 'CSPL_RECEIVED_CALLS']).sum().reset_index()

In [6]:
#Extract the time slot from date
def get_time(date):
    time = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').time()
    return time.hour * 3600 + time.minute * 60 + time.second

def get_month(date):
    month = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').month
    return month

#Assign a number to the day of the week
day_to_num_dict = {j:i for i,j in enumerate(['Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi','Dimanche'])}

training_data['TIME'] = training_data.DATE.map(get_time)
training_data['MONTH'] = training_data.DATE.map(get_month)
training_data['WEEK_DAY'] = training_data.DAY_WE_DS.map(day_to_num_dict)
training_data['NIGHT'] = (training_data.TPER_TEAM == "Nuit") * 1

# Remove obsolete columns
training_data = training_data[[col for col in training_data.columns if not col in ['DATE','DAY_WE_DS','TPER_TEAM']]]

In [7]:
#Convert the different ASS_ASSIGNMENTs to booleans
for value in submission.ASS_ASSIGNMENT.unique():
    training_data["ASS_ASSIGNMENT_"+value] = (training_data.ASS_ASSIGNMENT == value) * 1
    
# Remove obsolete column
training_data = training_data[[col for col in training_data.columns if not col == 'ASS_ASSIGNMENT']]

In [9]:
training_data.head()

,WEEK_END,CSPL_RECEIVED_CALLS,TIME,MONTH,WEEK_DAY,NIGHT,ASS_ASSIGNMENT_CAT,ASS_ASSIGNMENT_Téléphonie,ASS_ASSIGNMENT_Tech. Inter,ASS_ASSIGNMENT_Tech. Axa,ASS_ASSIGNMENT_Services,ASS_ASSIGNMENT_Regulation Medicale,ASS_ASSIGNMENT_RENAULT,ASS_ASSIGNMENT_Nuit,ASS_ASSIGNMENT_SAP,ASS_ASSIGNMENT_Japon,ASS_ASSIGNMENT_Gestion Renault,ASS_ASSIGNMENT_Gestion Amex,ASS_ASSIGNMENT_Gestion - Accueil Telephonique,ASS_ASSIGNMENT_Gestion,ASS_ASSIGNMENT_Domicile,ASS_ASSIGNMENT_Crises,ASS_ASSIGNMENT_Médical,ASS_ASSIGNMENT_Tech. Total,ASS_ASSIGNMENT_Mécanicien,ASS_ASSIGNMENT_Gestion Relation Clienteles,ASS_ASSIGNMENT_Manager,ASS_ASSIGNMENT_Gestion Clients,ASS_ASSIGNMENT_Gestion DZ,ASS_ASSIGNMENT_RTC,ASS_ASSIGNMENT_CMS,ASS_ASSIGNMENT_Prestataires,ASS_ASSIGNMENT_Gestion Assurances
0,1,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## View main statistics

In [10]:
training_data.describe()

,WEEK_END,CSPL_RECEIVED_CALLS,TIME,MONTH,WEEK_DAY,NIGHT,ASS_ASSIGNMENT_CAT,ASS_ASSIGNMENT_Téléphonie,ASS_ASSIGNMENT_Tech. Inter,ASS_ASSIGNMENT_Tech. Axa,ASS_ASSIGNMENT_Services,ASS_ASSIGNMENT_Regulation Medicale,ASS_ASSIGNMENT_RENAULT,ASS_ASSIGNMENT_Nuit,ASS_ASSIGNMENT_SAP,ASS_ASSIGNMENT_Japon,ASS_ASSIGNMENT_Gestion Renault,ASS_ASSIGNMENT_Gestion Amex,ASS_ASSIGNMENT_Gestion - Accueil Telephonique,ASS_ASSIGNMENT_Gestion,ASS_ASSIGNMENT_Domicile,ASS_ASSIGNMENT_Crises,ASS_ASSIGNMENT_Médical,ASS_ASSIGNMENT_Tech. Total,ASS_ASSIGNMENT_Mécanicien,ASS_ASSIGNMENT_Gestion Relation Clienteles,ASS_ASSIGNMENT_Manager,ASS_ASSIGNMENT_Gestion Clients,ASS_ASSIGNMENT_Gestion DZ,ASS_ASSIGNMENT_RTC,ASS_ASSIGNMENT_CMS,ASS_ASSIGNMENT_Prestataires,ASS_ASSIGNMENT_Gestion Assurances
count,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000,15158.000000
mean,0.284404,0.544333,5378.981396,2.350310,2.983903,1,0.002309,0.052645,0.052448,0.047434,0.052579,0.045125,0.052514,0.052579,0.052250,0.052579,0.050402,0.041298,0.051788,0.052579,0.052579,0.049479,0.052645,0.033514,0.012139,0.014646,0.020781,0.026587,0.018208,0.011809,0.003694,0.003826,0.041562
std,0.451145,2.339408,3851.110218,1.105501,1.993293,0,0.047998,0.223332,0.222935,0.212572,0.223200,0.207584,0.223068,0.223200,0.222537,0.223200,0.218781,0.198986,0.221606,0.223200,0.223200,0.216873,0.223332,0.179980,0.109509,0.120134,0.142656,0.160877,0.133708,0.108029,0.060671,0.061741,0.199593
min,0.000000,0.000000,0.000000,1.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1800.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,5400.000000,2.000000,3.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,9000.000000,3.000000,5.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,33.000000,12600.000000,4.000000,6.000000,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


##  A simple predictor

Lets try building a tree-based boosting predictor with very few attributes just to see how it goes.
This predictor will only predict the number of calls received during a given time stamp (e.g. in a 30 minutes slot) and day of week.

Note that the date is not relevant for regression, but we can extract some relevant information from it: day of the week, time slot, and if it is a week-end or not.

Also, for some reason the data for a given ASS_ASSIGNMENT and DATE is sometimes split, so we have to aggregate it.

In [11]:
output_cols = ['CSPL_RECEIVED_CALLS']
input_cols = [col for col in training_data.columns if not col in output_cols]

In [12]:
#Now create the gradient boosting regressor

X_train, X_test, y_train, y_test = train_test_split(training_data[input_cols],  training_data[output_cols].values.ravel())

est = GradientBoostingRegressor()

tuned_parameters = {'loss' : ['ls', 'lad', 'huber', 'quantile'],'n_estimators':[30],'learning_rate': [0.3,0.5,0.7], 'subsample': [1.0],
                     'min_samples_split':[1,3],'min_samples_leaf':[1,2],
                     'max_depth':[3,5],'max_features':['auto']
                    }
                     

clf = RandomizedSearchCV(est, tuned_parameters, cv=5,n_jobs=-1,n_iter=18,verbose=1)

clf.fit(X_train, y_train)

est_temoin=GradientBoostingRegressor()
est_temoin.fit(X_train,y_train)

print(clf.best_params_)
best_est=clf.best_estimator_

#Plot CV error (this is squared loss, which will be used to evaluate our performance in the leaderboard)

#Sur 10.000 lignes best_estimator est souvent moins bon que le temoin.. Par contre la différence est nette
# quand on utilise toutes les données.
#The higher the score the better.
print("Best estimator : %.4f" %best_est.score(X_test,y_test))
print("Temoin : %.4f" %est_temoin.score(X_test,y_test))



Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   21.0s finished


{'n_estimators': 30, 'learning_rate': 0.3, 'max_features': 'auto', 'loss': 'huber', 'max_depth': 5, 'subsample': 1.0, 'min_samples_leaf': 2, 'min_samples_split': 3}
Best estimator : 0.8506
Temoin : 0.8485


# Prediction and submission

In [42]:
test_data = submission.copy()

def get_weekday(date):
    return dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').weekday()

test_data['TIME'] = test_data.DATE.map(get_time)
test_data['MONTH'] = test_data.DATE.map(get_month)
test_data['WEEK_DAY'] = test_data.DATE.map(get_weekday)
test_data['NIGHT'] = (np.logical_or(test_data.TIME >= (23*3600 + 30*60),
                                    test_data.TIME <  (7*3600  + 30*60))) * 1
test_data['WEEK_END'] = test_data.WEEK_DAY.isin([5, 6]) * 1

# Convert the different ASS_ASSIGNMENTs to booleans
for value in submission.ASS_ASSIGNMENT.unique():
    test_data["ASS_ASSIGNMENT_"+value] = (test_data.ASS_ASSIGNMENT == value) * 1

test_data = test_data[input_cols]

In [43]:
submission.prediction = best_est.predict(test_data)
submission.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-01-03 00:00:00.000,CAT,0.167185
1,2012-01-03 00:00:00.000,Téléphonie,0.757762
2,2012-01-03 00:00:00.000,Tech. Inter,0.009142
3,2012-01-03 00:00:00.000,Tech. Axa,-0.003266
4,2012-01-03 00:00:00.000,Services,0.328637


In [44]:
# Write prediction to csv
submission.to_csv("data/output.txt", sep='\t', index=False)